# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [38]:
DATA_FOLDER = '../../ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [51]:
#imports
import pandas as pd
import datetime
from dateutil.parser import parse
from os import listdir
from os.path import isfile, join

In [52]:
def get_files(country):
    path = DATA_FOLDER + "/ebola/" + country + "_data/"
    return [f for f in listdir(path) if isfile(join(path, f))]


In [53]:
def get_row_guinea(country, file):
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file)
    interesting = raw[['Description', 'Totals']].fillna(0)
    new_cases = float(interesting[interesting.Description == "Total new cases registered so far"]['Totals'].values[0])
    new_deaths = float(interesting[(interesting.Description == "New deaths registered today") | (interesting.Description == "New deaths registered")]['Totals'].values[0])
    
    return {'Country' : country, 'Date' : parse(date), 'NewCases' : new_cases, 'NewDeaths' : new_deaths}

In [54]:
def get_row_liberia(country, file):
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file)
    interesting = raw[['Variable', 'National']].fillna(0)
    new_cases = float(sum(interesting[[v.lower().startswith("new case/s") for v in interesting.Variable]]['National'].values))
    new_deaths = float(interesting[interesting.Variable == "Newly reported deaths"]['National'].values[0])
    
    return {'Country' : country, 'Date' : parse(date), 'NewCases' : new_cases, 'NewDeaths' : new_deaths}

In [55]:
def get_row_sl(country, file):
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file)
    interesting = raw[['variable', 'National']].fillna(0)
    new_cases_array = interesting[(interesting.variable == "new_suspected") | (interesting.variable == "new_probable") | (interesting.variable == "new_confirmed")]['National'].values
    new_cases = sum([float(val_str) for val_str in new_cases_array])
    total_deaths_array = interesting[(interesting.variable == "death_suspected") | (interesting.variable == "death_probable") | (interesting.variable == "death_confirmed")]['National'].values
    total_deaths = sum([float(val_str) for val_str in total_deaths_array])
    
    return {'Country' : country, 'Date' : parse(date), 'NewCases' : new_cases, 'TotalDeaths' : total_deaths}

In [56]:
rows_guinea = [get_row_guinea("guinea", file) for file in get_files("guinea")]

In [57]:
rows_liberia = [get_row_liberia("liberia", file) for file in get_files("liberia")]

In [58]:
rows_sl_total_deaths = [get_row_sl("sl", file) for file in get_files("sl")]
dic_sl_total_deaths = {}
for row in rows_sl_total_deaths:
    dic_sl_total_deaths[row['Date']] = row
    
rows_sl = []
for date, entry in dic_sl_total_deaths.items():
    date_before = date - datetime.timedelta(days=1)
    if date_before in dic_sl_total_deaths:
        copy = dict(entry)
        del copy['TotalDeaths']
        copy['NewDeaths'] = entry['TotalDeaths'] - dic_sl_total_deaths[date_before]['TotalDeaths']
        rows_sl.append(copy)

In [59]:
raw_dataframe = pd.DataFrame(columns=['Country', 'Date', 'NewCases', 'NewDeaths'])
for row in rows_guinea, rows_sl:
    raw_dataframe = raw_dataframe.append(row, ignore_index = True)
for row in rows_liberia:
    if row['Date'].month != 12: #December data is erroneous
        raw_dataframe = raw_dataframe.append(row, ignore_index = True)

In [60]:
dataframe = raw_dataframe.set_index(['Country', 'Date'])

In [61]:
dataframe_no_day = raw_dataframe
dataframe_no_day['Year'] = raw_dataframe['Date'].apply(lambda x: x.year)
dataframe_no_day['Month'] = raw_dataframe['Date'].apply(lambda x: x.month)
dataframe_no_day[['Country', 'Year', 'Month', 'NewCases', 'NewDeaths']].groupby(['Country', 'Year', 'Month']).mean()

NewCases  NewDeaths
Country Year Month                      
guinea  2014 8      25.800000   3.400000
             9      19.625000   3.562500
             10     34.000000  15.000000
liberia 2014 6       5.714286   2.000000
             7       8.545455   4.272727
             8      37.222222  23.222222
             9      63.833333  36.041667
             10     45.560000  28.040000
             11     26.466667  13.466667
sl      2014 8      25.315789   6.947368
             9      42.142857  21.964286
             10     73.652174  23.391304
             11     70.588235  16.352941
             12     39.000000  30.000000

In [10]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here